# code to collect knowledge about Worl;d dye variety database


In [43]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import requests 
import re
import json 
import pickle 

In [44]:
# gather a list of links for a given moelcule

# collect all links in the list for basic dyes 
def linklist(type,base, pages):
    linklist = []
    # gather data for first page
    p1 = base
    page = requests.get(p1)
    soup = BeautifulSoup(page.content, 'html.parser')
    links=soup.find_all('a', {'href': re.compile(r'{}(-)(.*)(-)(\d+).html(\Z)'.format(type))})

    for link in links[0:5]:
        linklist.append(link.get('href'))
  

    for i in range(2,pages):
        p1 = base+'/page/{}'.format(i)
        page = requests.get(p1)
        soup = BeautifulSoup(page.content, 'html.parser')
        links=soup.find_all('a', {'href': re.compile(r'{}(-)(.*)(-)(\d+).html(\Z)'.format(type))})

        for link in links[0:5]:
            linklist.append(link.get('href'))
    return linklist

In [4]:
def GetInfo(link):
    page = requests.get(link)
    soup = BeautifulSoup(page.content, 'html.parser')
    #test for case with lightfastness
    data = []
    rows = soup.find_all('tr')
    img = soup.find_all('img')

    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols]) 
    test = soup.find_all(class_="entrytext")
    text = test[0].getText()    
        
    CAS = re.search("CAS Registry Number:(\d+-\d+-\d+)", text)
    Mol = re.search("Molecular Structure:.*", text)
    name=re.search("Name:.*", text)
    formula = re.search("Molecular Formula:.*", text)
    weight =re.search("Molecular Weight:.*", text)
    creation = re.search("Manufacturing Methods:.*", text)
    Props= re.search("Properties and Applications:.*", text)
    suppliers= re.search("List of Suppliers:.*", text)

    try:
        CAS = CAS[0]
    except:
        CAS = None

    try:
        Mol = Mol[0]
    except:
        Mol = None

    try:
        name = name[0]
    except:
        name = None

    try:
        formula = formula[0]
    except:
        formula = None

    try:
        weight = weight[0]
    except:
        weight = None

    try:
        creation = creation[0]
    except:
        creation = None

    try:
        Props = Props[0]
    except:
        Props = None

    try:
        suppliers = suppliers[0]
    except:
        suppliers = None

        
    info = {'CAS':CAS,'Mol':Mol,'name':name,'formula':formula,'weight':weight,'methods':creation,'properties':Props,
           'suppliers':suppliers}



      #print(match[0])
      #print(data)

    stuff = {'verbal':info,'numerical':data}

    return stuff


In [5]:
def Collect(name, link, number):
    LinkList = linklist(name,link,number)
    DataDump = []
    for link in LinkList:

        try:
            DyeData = GetInfo(link) 
            DataDump.append(DyeData)
        except:
            continue 
        

    # convert the data into a json and save 

    Dyes = json.dumps(DataDump)
    with open("{}Dyes.json".format(name), "w") as outfile:
        outfile.write(Dyes)


In [6]:
# acid dyes


Collect('acid','https://www.worlddyevariety.com/category/acid-dyes',160)

KeyboardInterrupt: 

In [7]:
#160
LinkList = linklist('acid','https://www.worlddyevariety.com/category/acid-dyes',10)
DataDump = []
for link in LinkList:
    try:
        DyeData = GetInfo(link) 
        DataDump.append(DyeData)
    except:
        continue 
        

        
        
Dyes = json.dumps(DataDump)
with open("{}Dyes_img.json".format('acid'), "w") as outfile:
    outfile.write(Dyes)        
#Dyes = pickle.dump(DataDump,outfile)
#with open("{}Dyes.pkl".format('acid'), "w") as outfile:
#    pickle.dump(DataDump,outfile)


In [87]:
DataDump[0]['verbal']['Cas'][0]

'CAS Registry Number:18472-87-2'

In [129]:
GetInfo('https://www.worlddyevariety.com/food-dyes/food-green-2.html')

{'verbal': {'CAS': 'CAS Registry Number:5141-20-8',
  'Mol': 'Molecular Structure: triarylmethane class',
  'name': 'Name:C.I.Food green 2,C.I.42095',
  'formula': 'Molecular Formula:C37H34N2Na2O9S3',
  'weight': 'Molecular Weight:  792.85',
  'methods': None,
  'properties': 'Properties and Applications:  green. Soluble in water for blue light green, hardly soluble in ethanol (green). The strong sulfuric acid in orange, dilute yellowish later. The dye solution to join hydrogen oxidation, nearly colorless, with dark purple precipitation. And C.I. Acid Green 5 the same chemical structure.',
  'suppliers': 'List of Suppliers:'},
 'numerical': []}

# trying to do these with multi processing

In [132]:
Collect('food','http://www.worlddyevariety.com/category/food-dyes',15)

In [105]:
from multiprocessing import Pool

In [ ]:
def f(x):
    return x*x

if __name__ == '__main__':
    with Pool(5) as p:
        print(p.map(f, [1, 2, 3]))

In [117]:
#process parameters 

ll = [('acid','https://www.worlddyevariety.com/category/acid-dyes',160),
    ('azoic','http://www.worlddyevariety.com/category/azoic-dyes',22),
     ('basic','http://www.worlddyevariety.com/category/basic-dyes',37),
     ('direct','http://www.worlddyevariety.com/category/direct-dyes',154),
     ('disperse','http://www.worlddyevariety.com/category/disperse-dyes',110),
     ('food','http://www.worlddyevariety.com/category/food-dyes',15),
     ('mordant','http://www.worlddyevariety.com/category/mordant-dyes',51),
     ('reactive','http://www.worlddyevariety.com/category/reactive-dyes',67),
     ('solvent','http://www.worlddyevariety.com/category/solvent-dyes',90),
     ('sulfur','http://www.worlddyevariety.com/category/sulphur-dyes',45),
     ('vat','http://www.worlddyevariety.com/category/vat-dyes',53)]

In [ ]:
starmap(Collect,ll)

In [133]:
with Pool(11) as p:
    p.starmap(Collect,ll)

In [134]:
1

1

In [9]:
page  = requests.get('https://www.worlddyevariety.com/acid-dyes/acid-yellow-4.html')

In [10]:
soup = BeautifulSoup(page.content, 'html.parser')

In [12]:
soup.find_all('img',{href:'class'})

[<img alt="Subscribe to RSS feed" src="https://www.worlddyevariety.com/wp-content/themes/worlddyevariety/images/rss-feed-icon.jpg"/>,
 <img alt="必应染料-您身边的染料专家" height="100" longdesc="http://www.biyingdye.com/" src="https://ww3.sinaimg.cn/large/005CMeJsgw1enzugvyl3og30qe02smxi.gif" width="950"/>,
 <img alt="380.35, C.I.18695, C.I.Acid Yellow 4, C16H13N4NaO4S, CAS 6359-74-6, Tartrazine J, Yellow RN" class="size-full wp-image-4902" height="172" src="https://www.worlddyevariety.com/wp-content/uploads/2012/05/Acid-Yellow-4.gif" title="380.35, C.I.18695, C.I.Acid Yellow 4, C16H13N4NaO4S, CAS 6359-74-6, Tartrazine J, Yellow RN" width="400"/>]

In [15]:
mydivs = soup.find_all("img", {"class": "size-full wp-image-4902"})

In [16]:
mydivs

[<img alt="380.35, C.I.18695, C.I.Acid Yellow 4, C16H13N4NaO4S, CAS 6359-74-6, Tartrazine J, Yellow RN" class="size-full wp-image-4902" height="172" src="https://www.worlddyevariety.com/wp-content/uploads/2012/05/Acid-Yellow-4.gif" title="380.35, C.I.18695, C.I.Acid Yellow 4, C16H13N4NaO4S, CAS 6359-74-6, Tartrazine J, Yellow RN" width="400"/>]

In [17]:
mydivs = soup.find_all("img", {"class": "size-full"})

In [35]:
mydivs[0]['src']

'https://www.worlddyevariety.com/wp-content/uploads/2012/05/Acid-Yellow-4.gif'

In [39]:
test=mydivs[0]['alt']

In [64]:
cas = re.search('\d+-\d+-\d+',test)

In [65]:
cas[0]

'6359-74-6'

# place all the dyes in a txt file


In [47]:
grand_links= []

grand_links.append(linklist('acid','https://www.worlddyevariety.com/category/acid-dyes',160))
grand_links.append(linklist('azoic','http://www.worlddyevariety.com/category/azoic-dyes',22))
grand_links.append(linklist('basic','http://www.worlddyevariety.com/category/basic-dyes',37))
grand_links.append(linklist('direct','http://www.worlddyevariety.com/category/direct-dyes',154))
grand_links.append(linklist('disperse','http://www.worlddyevariety.com/category/disperse-dyes',110))
grand_links.append(linklist('food','http://www.worlddyevariety.com/category/food-dyes',15))
grand_links.append(linklist('mordant','http://www.worlddyevariety.com/category/mordant-dyes',51))
grand_links.append(linklist('reactive','http://www.worlddyevariety.com/category/reactive-dyes',67))
grand_links.append(linklist('solvent','http://www.worlddyevariety.com/category/solvent-dyes',90))
grand_links.append(linklist('vat','http://www.worlddyevariety.com/category/vat-dyes',53))

In [51]:
with open('/home/matt/Proj/Dyes/Dyebrary/DyeScraping/DyeLinks.txt',mode='w') as f:
     for item in grand_links:
        # write each item on a new line
        f.write("%s\n" % item)
        



In [66]:
import requests
import re
from bs4 import BeautifulSoup
def GetPicCAS(link):
    page= requests.get(link)
    soup= soup.find_all("img", {"class": "size-full"})
    img_link =  soup[0]['src']
    CAS = re.search('\d+-\d+-\d+',soup[0]['alt'])

    return CAS, img_link
    
